# Chicken and egg in molecular metadynamics

### Create protein folding landmarks from scratch


The whole workflow is implemented as a single notebook to allow batch run. It is structured to these major steps:

1. generate random landmarks by twisting the input structrure backbone dihedrals and steep-descend minimization of the result to avoid side chain clash etc.
1. compute low-dimensional embedding (isomap) of the landmarks to create collective variables
1. train a neural networks to estimate the colvars from the structure, implement the resulting network as plumed input
1. run both vanilla molecular dynamics and the metadynamics to compare the trajectories


In [ ]:
# we need all this fancy shit
import anncolvar

import os
import shutil
from contextlib import redirect_stdout
import re

import concurrent.futures

import numpy as np
import math
from scipy.sparse import coo_matrix

from pyDOE import lhs
from sklearn.utils.graph_shortest_path import graph_shortest_path
from sklearn.manifold import MDS

import PeptideBuilder as pb
import Bio.PDB as pdb
import Bio.SeqUtils as sequtil

import mdtraj as md

import matplotlib.pyplot as plt
import nglview as nv

from xvg import read_xvg

In [ ]:
# set the defaults 
# for batch processing, those can be set in config.py so that the notebook itself needn't be modified

# input PDB name (to be downloaded) or a provided file
#pdbid = '1L2Y'

pdbfile = "1L2Y.pdb"

# number of steps to twist a dihedral when generating landmarks
nsteps = 12

# number of iterations of landmark generation; alltogether niter * nsteps random conformers are generated
niter = 200

# number of steps of the production MD run
# md.mdp template sets 2 fs step
mdsteps = 2500000  # 5 ns, this is too short, for testing only!

# bounding box size to add for mininimization and 
minbox = 1.5
mdbox = 1.5


# count available cores -- check the output and set ncores to something else if necessary

# TODO: capture PBS settings when running in batch

if os.environ.get('OMP_NUM_THREADS') is None:
    ncores = int(os.popen('./ncores.sh').read())
    print('OMP_NUM_THREADS not set, using all (%d) available cores' % ncores)
else:
    ncores = int(os.environ.get('OMP_NUM_THREADS'))
    print('Using OMP_NUM_THREADS = %d cores' % ncores)

# XXX: poor man approach, expected to be tuned in config.py
ntomp = 4
ntmpi = ncores // ntomp

try:
    exec(open('config.py').read())
    !cat config.py
except FileNotFoundError:
    print('config.py not found, hope it\'s OK')

## 1 Generate landmarks by random twisting PDB structure

### 1.1 Load structure from PDB

Load a PDB file, store it locally, create a matching workdir, and look at the PDB content

In [ ]:
try:
    workdir
    raise Exception("This cell should be run only once (workdir = %s)" % workdir)
except NameError:
    pass

basedir=os.getcwd()


# load the PDB file, create workdir, set global variables to be used later
do_load = False
try:
    pdbid
    do_load = True
    pdbfile = pdbid + '.pdb'

except NameError:
    pdbid = os.path.splitext(os.path.basename(pdbfile))[0]
    
workdir=os.path.join(basedir,pdbid)

if not os.path.exists(workdir):
    os.mkdir(workdir)

os.chdir(workdir)

if do_load:
    pdbl = pdb.PDBList()
    pdbl.retrieve_pdb_file(pdbid)
    shutil.move(pdbid + ".ent", pdbid + ".pdb")
else:
    shutil.copy(os.path.join(basedir,pdbfile),os.path.join(workdir,pdbfile))

In [ ]:
# preprocess the file with gromacs to get consistent atom naming and numbering
!gmx pdb2gmx -f {pdbfile} -o {pdbid}-new.pdb -water tip3p -ff amber94 -ignh && \
mv {pdbid}-new.pdb {pdbfile}

In [ ]:
# inspect the loaded file
# it must be a sane structure, no missing heavy atoms and/or hydrogens etc., 
# suitable as the starting point of usual MD protocol
os.chdir(basedir)
v = nv.NGLWidget()
v.add_component(os.path.join(workdir,pdbfile))
v.clear()
v.add_representation('cartoon', selection='all')

os.chdir(workdir)
v

### 1.2 Generate randomly twisted conformations

$\phi$ and $\psi$ backbone dihedral angles of all but first and last residue of the loaded structure are twisted randomly.

Systematic approach (e.g. 30 degree sampling of all angles) would yield too many conformations.
Instead we use random latin hypercube sampling to get uniform coverage of all values of all angles.

Empirically, running 10 times no. of residues (`niter` parameter bellow) seems to be sufficient to cover the whole conformational space while keeping number of landmarks still reasonable.

Expect approx. 1 s per 300 residues. It is worth to inspect some of the outputs visually (the following cell).

In [ ]:
p = pdb.PDBParser()
instruct = p.get_structure('in',pdbfile)

# XXX: assuming one model and one chain, the method would be rather weird for more

resl = list(map(lambda r: sequtil.seq1(r.get_resname()),instruct.get_residues()))
nres = len(resl)

out='conf%d.pdb'

# make it really reproducible
np.random.seed(123456789)
    
def random_twist(iter):    
    phi = lhs(nres - 2, nsteps)
    psi = lhs(nres - 2, nsteps)
    outf = pdb.PDBIO()

    for s in range(nsteps):
        first = pb.Geometry.geometry(resl[0])
        struct = pb.initialize_res(first)
        
        for r in range(1,nres-1):
            if resl[r] == 'P':
                pb.add_residue(struct,resl[r])
            else:
                pb.add_residue(struct,resl[r],phi[s][r-1]*360,psi[2][r-1]*360)
                
        pb.add_residue(struct,resl[nres-1])
            
        fn = out % (iter * nsteps + s + 1)
        outf.set_structure(struct)
        outf.save(fn)
        
# XXX: better with hyperthreading but we don't want to eat up 2x cores when running in batch mode
with concurrent.futures.ProcessPoolExecutor(max_workers=ncores) as executor:
    for _ in executor.map(random_twist,range(niter)):
        pass





In [ ]:
tr = md.load([ "conf%d.pdb" % i for i in range(1,nsteps*niter+1)])
idx=tr[0].top.select("name CA")
tr.superpose(tr[0],atom_indices=idx)

v=nv.show_mdtraj(tr)
v.clear()
v.add_representation("licorice")
v

### 1.3 Minimize the generated structures

Run Gromacs steepest descend energy minimization in vacuo on all the generated structures. This is sufficient to fix colliding sidechains etc. while not changing the backbone dihedrals, hence preserving the conformational space coverage.

Expect approx. 25 structures per minute per core in case of small protein like trpcage (1L2Y).

In [ ]:
# the most likely minimization parameters to change; rest is in the template file
minim_mdp = '''
emtol       = 500.0        ; Stop minimization when the maximum force is lower (kJ/mol/nm)
emstep      = 0.05          ; Minimization step size
nsteps      = 5000         ; Maximum number of (minimization) steps to perform
'''

template = os.path.join(basedir,'minim.mdp.template')

!cp {template} minim.mdp
f=open('minim.mdp','a')
f.write(minim_mdp)
f.close()

minim = os.path.join(basedir,'minim.sh')
!bash {minim} {ncores} {minbox}

In [ ]:
# filter the results to the reasonable ones only

conflist = []
energies = []
maxenergy = 1e8

for i in range(1,nsteps*niter+1):
    try:
        with open('conf%d.minen' % i) as ef:
            l = ef.readline()
            _,energy = l.split()
            energy = float(energy)
    except FileNotFoundError:
        print(i, "not found, something went wrong")
        continue
        
    if energy < maxenergy:
        conflist.append(i)
        energies.append(energy)
    else:
        print(i,"energy too high:", energy, 'ignoring')
        
print('remaining conformers', len(conflist))

### 1.4 Inspect the results

Minimized structures are merged into virtual trajectory and displayed as animation.

Histograms of their radius of gyration and energies (following cells) gives some evidence on conformational space coverage.

In [ ]:
# we may get varying protonation with weird generated conformers, hence drop hydrogens here to avoid confusion
frames = []
for i in conflist:
    fn = "conf%d-min.gro" % i
    if os.path.isfile(fn):
        one = md.load(fn)
        heavy_idx = one[0].top.select("element != H")
        one.atom_slice(heavy_idx,inplace=True)
        frames.append(one)
        
tr=md.join(frames)

tr[0].center_coordinates()
idx=tr[0].top.select("name CA")
tr.superpose(tr[0],atom_indices=idx)

v=nv.show_mdtraj(tr,gui=True)
v.clear()
v.add_representation("licorice")
v

In [ ]:
rgs=md.compute_rg(tr)

plt.figure(figsize=(15,5))
plt.hist(rgs,30)
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
plt.hist(energies,30)
plt.show()

In [ ]:
# save superposed landmarks as Gromacs trajectory
tr.save_pdb('landmarks.pdb')
tr.save_xtc('landmarks.xtc')
tr[0].save_pdb('landmark1.pdb')

## 2. Compute isomap projection of the landmarks

In [ ]:
# number of nearest neighbours to consider (aka _k_)
neighs = 5
# targed no. of dimensions 
dims = 2

try:
    tr
except NameError:
    tr = md.load('landmarks.pdb')

In [ ]:
# nconf = niter * nsteps
nconf = len(tr)

# compute all-to-all RMSD and select _k_ closest neighbours
row=[]
col=[]
dat=[]

for i in range(nconf):
    d = md.rmsd(tr,tr,frame=i)
    d[range(i+1)] = np.inf
    for _ in range(neighs):
        j = np.argmin(d)
        if d[j] < np.inf:
            row.append(i)
            col.append(j)
            dat.append(d[j])
            row.append(j)
            col.append(i)
            dat.append(d[j])
            d[j] = np.inf

# store results in sparse matrix
dist = coo_matrix((dat,(row,col)),shape=(nconf,nconf)) 

# check sanity
print("conformations (original dimensions): ", nconf)
print("non-zero distances: ", dist.getnnz())

In [ ]:
# isomap itself: compute shortest paths in the k-neighbours graph, 
# and multi-dimensional scaling on the resulting all-to-all distances
sp = graph_shortest_path(dist,directed=False)
mds = MDS(n_components=dims,dissimilarity='precomputed',n_jobs=ncores)
emb = mds.fit_transform(sp)

In [ ]:
plt.figure(figsize=(12,8))
plt.scatter(*emb.transpose(),marker='.')
plt.show()

In [ ]:
# XXX: assumes negative min and positive max
embmin=np.min(emb,axis=0)*1.2
embmax=np.max(emb,axis=0)*1.2

In [ ]:
# save collective variables
np.savetxt('colvar.txt',emb)

## 3 Train the neural net

Create artificial neural network and train it to produce the above isomap embedding from superposed heavy atom coordinates. The ANN is encoded in `plumed.dat` to be used by metadynamic run later.

Technically, this is done for both coordinates of the embedding independently, the resulting `plumed.dat` files are merged.

Uses [Anncolvar](https://github.com/spiwokv/anncolvar).

In [ ]:
# run anncolvar twice, for each isomap coordinate independently
epochs = 500

# anncolvar defaults
nlayers = 1
layers = [32, 0, 0]
actfun = ['sigmoid','linear','linear']
optim = 'adam'
loss = 'mean_squared_error'
batch = 256


rg = md.compute_rg(first)

# XXX magic -- seems to be safe
shift = rg[0] * 5
box = shift * 2

!gmx editconf -f landmark1.pdb -o landmark1-box.pdb -translate {shift} {shift} {shift} -box {box} {box} {box} -c >editconf.log 2>&1

with open('landmark1-box.pdb') as boxf, open('reference.pdb','w') as ref:
    lines = boxf.readlines()
    for l in lines:
        if l[:4] == 'ATOM':
            newi = heavy_idx[int(l[4:11])-1]+1
            ref.write('ATOM%7d' % newi)
            ref.write(l[11:])
        else:
            ref.write(l)

# XXX: too much stdout 
for col in [1,2]:
    with open("anncolvar-%d.log" % col,"w") as log, redirect_stdout(log):
        anncolvar.anncollectivevariable('landmarks.xtc','reference.pdb','colvar.txt',col,
                                    box,box,box,.1,0,0,
                                    nlayers,*layers,
                                    *actfun,
                                    optim,loss,epochs,batch,
                                    '','','plumed%d.dat' % col)
            
!tail editconf.log anncolvar-[12].log

In [ ]:
# merge plumed[12].dat from the previous cell

# XXX entirely
with open('plumed.dat','w') as fout:
    for col in [1,2]:
        with open('plumed%d.dat' % col) as fin:
            for line in fin:
                '''not necessary, reference.pdb is already renumbered
                w = line.split()
                if w[1] == 'POSITION':
                    a = w[2].split('=')
                    line = w[0] + (' POSITION ATOM=%d ' % (heavy_idx[int(a[1])-1]+1)) + ' '.join(w[3:]) + '\n'
                '''
                if re.match('[pl][0-9_rxyz]',line):
                    line = re.sub('[pl][0-9_rxyz]','cv%d_\g<0>' % col,line)
                    fout.write(line)
                elif col == 1 and not re.match('PRINT',line):
                    fout.write(line)
  
    # XXX: hardcoded
    fout.write('PRINT ARG=cv1_l2r,cv2_l2r STRIDE=100 FILE=COLVAR\n')
    fout.write('METAD ARG=cv1_l2r,cv2_l2r SIGMA=0.1,0.1 HEIGHT=1.0 PACE=1000 BIASFACTOR=15 TEMP=300 LABEL=restraint')
    fout.write(' GRID_MIN=%f,%f GRID_MAX=%f,%f GRID_BIN=150,150\n' % (*embmin,*embmax))

## 4. Run MD

Run quite standard molecular dynamics protocol, adapted from [Lysosome tutorial](http://www.mdtutorials.com/gmx/lysozyme/index.html), i.e. solvate, add counterions, minimize, equilibrate, and run production.

Preparation phases are common, then we run vanilla and metadynamic simulations to compare the results.

### 4.1 Prepare, minimize, and equilibrate

In [ ]:
# elementary preparation

# XXX hardcoded defaults for the time being, replace with template eventually

os.chdir(basedir)
!cp ions.mdp minim-sol.mdp {workdir}
os.chdir(workdir)

!gmx pdb2gmx -f {pdbfile} -o {pdbid}.gro -water tip3p -ff amber94 -ignh && \
gmx editconf -f {pdbid}.gro -o {pdbid}-box.gro -c -d {mdbox} -bt dodecahedron && \
gmx solvate -cp {pdbid}-box.gro -cs spc216.gro -o {pdbid}-solv.gro -p topol.top && \
gmx grompp -f ions.mdp -c {pdbid}-solv.gro -p topol.top -o ions.tpr && \
echo 13 | gmx genion -s ions.tpr -o {pdbid}-ions.gro -p topol.top -pname NA -nname CL -neutral

In [ ]:
# minimize with steepest descend
!gmx grompp -f minim-sol.mdp -c {pdbid}-ions.gro -p topol.top -o em.tpr &&\
gmx mdrun -v -deffnm em -ntmpi {ntmpi} -ntomp {ntomp} -pin on &&\
echo '10\
' | gmx energy -f em.edr -o em.xvg

In [ ]:
x,y=read_xvg(os.path.join(workdir,'em.xvg'))

plt.figure(figsize=(15,5))
plt.plot(x,y)
plt.grid()
plt.xlabel('step')
plt.ylabel('potential (kJ/mol)')
plt.title('Energy minimization')

plt.show()

In [ ]:
# isothermal - isochoric equilibration
!cp {basedir}/nvt.mdp .

!gmx grompp -f nvt.mdp -c em.gro -r em.gro -p topol.top -o nvt.tpr && \
gmx mdrun -ntomp {ntomp} -ntmpi {ntmpi} -pin on -deffnm nvt && \
echo '16\
' | gmx energy -f nvt.edr -o temp.xvg

In [ ]:
x,y=read_xvg(os.path.join(workdir,'temp.xvg'))

plt.figure(figsize=(15,5))
plt.plot(x,y)
plt.grid()
plt.xlabel('time (ps)')
plt.ylabel('temperature (K)')
plt.title('isothermal-isochoric equilibration')
plt.show()

In [ ]:
# isothermal - isobaric equilibration
!cp {basedir}/npt.mdp .

!gmx grompp -f npt.mdp -c nvt.gro -r nvt.gro -t nvt.cpt -p topol.top -o npt.tpr && \
gmx mdrun -ntomp {ntomp} -ntmpi {ntmpi} -pin on -deffnm npt && \
echo '18\
' | gmx energy -f npt.edr -o press.xvg && \
echo '24\
' | gmx energy -f npt.edr -o dens.xvg

In [ ]:
xp,yp=read_xvg(os.path.join(workdir,'press.xvg'))
xd,yd=read_xvg(os.path.join(workdir,'dens.xvg'))

plt.figure(figsize=(15,8))
plt.subplot(211)
plt.plot(xp,yp)
plt.title('isothermal-isobaric equilibration')
plt.grid()
#plt.xlabel('time (ps)')
plt.ylabel("pressure (bar)")


plt.subplot(212)
plt.xlabel('time (ps)')
plt.ylabel('density (kg/m3)')
plt.grid()
plt.plot(xd,yd)
plt.show()

### 4.2 Run vanilla MD

In [ ]:
!cp {basedir}/md.mdp.template md.mdp
with open('md.mdp','a') as mdp:
    mdp.write("nsteps = %d\n" % mdsteps)

!gmx grompp -f md.mdp -c npt.gro -t npt.cpt -p topol.top -o md1.tpr && \
gmx mdrun -ntomp {ntomp} -ntmpi {ntmpi} -pin on -deffnm md1

In [ ]:
!echo 1 | gmx trjconv -f md1.xtc -s npt.gro -pbc nojump -o pbc1.xtc

tr1 = md.load_xtc('pbc1.xtc',top=pdbid+'.gro')
idx=tr1[0].top.select("name CA")
tr1.superpose(tr1[0],atom_indices=idx)
v = nv.show_mdtraj(tr1,gui=True)
v

In [ ]:
rmsd = md.rmsd(tr1,tr1)
rg = md.compute_rg(tr1)
plt.figure(figsize=(15,8))
plt.subplot(211)
plt.plot(rmsd)
plt.grid()
plt.ylabel('RMSD wrt. frame 0 (nm)')
plt.subplot(212)
plt.plot(rg)
plt.grid()
plt.ylabel('Radious of gyration (nm)')
plt.xlabel('time (10 ps steps)')
plt.show()

### 4.3 Run metadynamics

In [ ]:
!cp {basedir}/md.mdp.template md.mdp

# XXX: hack plumed.dat, FIT_TO_TEMPLATE TYPE=OPTIMAL is broken
!sed '/^FIT_TO_TEMPLATE/s/TYPE=OPTIMAL/TYPE=SIMPLE/' plumed.dat >plumed.dat.$$ && mv plumed.dat.$$ plumed.dat

with open('md.mdp','a') as mdp:
    mdp.write("nsteps = %d\n" % mdsteps)

!gmx grompp -f md.mdp -c npt.gro -t npt.cpt -p topol.top -o md2.tpr && \
gmx mdrun -ntomp {ntomp} -ntmpi {ntmpi} -pin on -deffnm md2 -plumed plumed

In [ ]:
!echo 1 | gmx trjconv -f md2.xtc -s npt.gro -pbc nojump -o pbc2.xtc

tr2 = md.load_xtc('pbc2.xtc',top=pdbid+'.gro')
idx=tr2[0].top.select("name CA")
tr2.superpose(tr2[0],atom_indices=idx)
v = nv.show_mdtraj(tr2,gui=True)
v

In [ ]:
rmsd = md.rmsd(tr2,tr2)
rg = md.compute_rg(tr2)
plt.figure(figsize=(15,8))
plt.subplot(211)
plt.plot(rmsd)
plt.grid()
plt.ylabel('RMSD wrt. frame 0 (nm)')
plt.subplot(212)
plt.plot(rg)
plt.grid()
plt.ylabel('Radious of gyration (nm)')
plt.xlabel('time (10 ps steps)')
plt.show()